# Classic Segmentation

<div class="custom-button-row">
    <a 
        class="custom-button custom-download-button" href="../../notebooks/4_python_basics/Intro_to_Python_II.ipynb" download>
        <i class="fas fa-download"></i> Download this Notebook
    </a>
    <a
    class="custom-button custom-download-button" href="https://colab.research.google.com/github/HMS-IAC/bobiac/blob/gh-pages/colab_notebooks/4_python_basics/Intro_to_Python_II.ipynb" target="_blank">
        <img class="button-icon" src="../../_static/logo/icon-google-colab.svg" alt="Open in Colab">
        Open in Colab
    </a>
</div>

In [1]:
# /// script
# requires-python = ">=3.10"
# ///

# Standard library imports (no need to declare in dependencies)

## Overview

This notebook covers the following steps in building a **classic segmentation pipeline**:

| Step | Concept | Why it matters |
|---------|---------|----------------|
| 0 | Setup | Import necessary packages |
| 1 | Loading an Image | Open tif image and display it |
| 2 | Filtering | Learn how to process images to improve thresholding results |
| 3 | Thresholding | Learn how to use thresholding to generate a binary mask |
| 4 | Labeling | Learn how to label binary masks |
| 5 | Mask Refinement | Learn how to use watershed segmentation to refine binary masks |
| 6 | Processing Many Images | Learn how to apply these processing steps to many images |


Each chapter has:

1. **Summary** – review core concepts from lecture.
2. ✍️ **Exercise** – _your turn_ to write code!

***

## 0. Setup

**Concept.**  
We are going to be using existing Python packages for classic segmentation, so we need to specify them at the beginning of our code. This is called specifying our **dependencies**. It's standard practice to specify all dependencies at the very beginning of your code.

For learning purposes, we will import everything here step by step, as it is introduced in the following chapters.

### ✍️ Exercise: Run the following code block to specify our dependencies

In [2]:
# specify dependencies
import ndv
import numpy as np
import tifffile
from scipy.ndimage import distance_transform_edt
from skimage.color import label2rgb
from skimage.feature import peak_local_max
from skimage.filters import gaussian, threshold_otsu
from skimage.measure import label
from skimage.morphology import binary_closing, remove_small_objects
from skimage.segmentation import watershed

***

## 1. Loading an Image

**Concept.**
To work with an image in Python, we need to specify where the image file is so that we can **read**, or load, it. Once the image file is read, we can **view** it and process it.  

### Specifying your file's path
Once you have found your file's path, you should assign it to a variable to make it easy to work with. Here's an example:
```python
file_path = '/Users/edelase/HMS Dropbox/Eva de la Serna/Eva_CITE_folder/projects/bobiac/lectures/classic_segmentation/img.tif'
```

**❗️CAUTION❗️** Be wary of spaces in folder names, as they sometimes cause terminal to add `\` or `/` to file directories where they should not be. It is best practice to always use `_` in folder names whenever you would have wanted to have a space.

**🗒️ NOTE:** Specifying file paths is a common task outside of image segmentation with Python, so some of you may have experience with this already. Note the terminology though. An individual file's location is a **path**. A folder's path containing an individual or multiple files is called a **directory**.

### ✍️ Exercise: Specify your image file's path and assign it to the variable `image_path`

In [6]:
image_path = "../../../_static/images/classic_seg/DAPI_wf_0.tif"

### Reading an image
There are many different ways to read image files in Python. In this lesson, we are using the Python package `tifffile`, to read .tif image files. In order to read an image with `tifffile`, we will need to import it, and then provide it with the image's path:
```python
import tifffile # add this to 0. Setup code block
raw_image = tifffile.imread(image_path)
```
`tifffile.imread()` will use that `image_path` you inputted to find your file and read it. It will then return the read file. Since we will be wanting to work with this file, we assign it to the variable `raw_image` for easy reference. 

### ✍️ Exercise: Run the following code to read the image 

In [7]:
raw_image = tifffile.imread(image_path)
print(raw_image.shape, raw_image.dtype)

(2304, 2304) float32


### Viewing the image
In Python, reading the image and viewing it are two separate actions. Now that we have read the image and assigned it to the variable `raw_image`, we can view it using `ndv`, which we will need to import in Setup. 

| Package Name | Description | How to import it | Documentation Link | 
|---------|---------|----------------| ----------------|
| ndv | Multi-dimensional image viewer | `import ndv` | [ndv](https://pypi.org/project/ndv/) |

We can use ndv to view `raw_image` as follows: 
```python
ndv.imshow(raw_image)
```
`ndv.imshow()` will use that `raw_image` you inputted to display your image. It will then return the image displayed in the ndv viewer. 

### ✍️ Exercise: Run the following code to view the image with `ndv`
Remember, we need to import ndv in Setup!

In [ ]:
ndv.imshow(raw_image)

In [ ]:
viewer = ndv.imshow(raw_image)

In [ ]:
viewer.widget().children[1].snapshot()

***

## 2. Filtering

**Concept.**  
**Filters** change image pixel values using a **mathematical operation** to smooth and reduce **noise** from images. They can help improve thresholding results. 

### Applying a filter to an image
There are many different filters we can apply. Here's a summary of the ones we covered in depth in lecture: 

| Filter Name | Description | How to import it | Documentation Link | 
|---------|---------|----------------| ----------------|
| mean filter | For a given kernel size, sums values in a list and and then divides by the total number of values | `from skimage.filters.rank import mean` | [skimage.filters.rank.mean](https://scikit-image.org/docs/0.25.x/api/skimage.filters.rank.html#skimage.filters.rank.mean) |
| Gaussian blur filter | For a given kernel size, multiply each value by a Gaussian profile weighting, then divide by the total number of values | `from skimage.filters import gaussian` | [skimmage.filters.gaussian](https://scikit-image.org/docs/dev/api/skimage.filters.html#skimage.filters.gaussian) |
| median filter | For a given kernel size, take the middle number in a sorted list of numbers | `from skimage.filters import median` | [skimage.filters.median](https://scikit-image.org/docs/dev/api/skimage.filters.html#skimage.filters.median) |


### ✍️ Exercise: Write code to apply a Gaussian blur filter to `raw_image` and assign it to the variable `filtered_image`
Remember - we need to import the gaussian function from skimage.filters in Setup!

In [9]:
filtered_image = gaussian(raw_image)

### ✍️ Exercise: Run the following code to view `filtered_image` with `ndv`

In [ ]:
ndv.imshow(filtered_image)

In [ ]:
viewer = ndv.imshow(filtered_image)

In [ ]:
viewer.widget().children[1].snapshot()

***

## 3. Thresholding

**Concept.**  
**Thresholding** is when we select a range of digital values, or **intensity values**, in the image. These selected values are how we define regions of the image we are interested in. 

### Defining a threshold
We need to define a minimum intensity cutoff which separates the **background** (what we don't care about) from the **foreground** (what we do care about). We can manually pick a an intensity value as this cutoff value like below:
```python
min_threshold = 50
```

However, manually changing the value assigned to `min_threshold` until we find an optimal intensity cutoff value is tedious and may vary between images in a dataset. Therefore, it is best practice to instead use established **thresholding algorithms** to automatically define an intensity cutoff value. `skimage.filters` contains many different types of thresholding algorithms, but from that we will be using the Otsu thresholding algorithm `threshold_otsu`. 

| Thresholding Algorithm | Description | How to import it | Documentation Link | 
|---------|---------|----------------| ----------------|
| Otsu thresholding | Returns threshold using Otsu's method | `from skimage.filters import threshold_otsu` | [threshold_otsu](https://scikit-image.org/docs/dev/api/skimage.filters.html#skimage.filters.threshold_otsu) |

We can use `threshold_otsu` from `skimage.filters` as follows: 
```python
min_threshold = threshold_otsu(filtered_image)
```
Here, `threshold_otsu()` will use that inputted `filtered_image` to calculate an intensity cutoff value. It will return the cutoff value assigned to the variable `min_threshold`. 

### ✍️ Exercise: Write code to calculate a threshold on `filtered_image` using Otsu's method
Remember - we need to import the threshold_otsu function from skimage.filters in Setup!

In [10]:
min_threshold = threshold_otsu(filtered_image)

### Generating a binary mask
We now want to use this cutoff value to generate a **binary mask**, which is an image that has only 2 pixel values: one corresponding to the background and one corresponding to the foreground. By generating the binary mask, we will be able to evaluate whether this `min_threshold` is a sufficient cutoff value. 

We can generate the binary mask by using the comparison operator `>`:
```python
binary_mask = filtered_image > min_threshold
```
Python will interpret this line of code by going pixel by pixel through `filtered_image` and assigning `True` values where a pixel is greater than `min_threshold` and assigning `False` values where a pixel is equal or less than `min_threshold`. The output will be the binary mask image, filled with `True` and `False`. Since this binary mask is something we will be working with, we should assign it a variable, such as `binary_mask`. 

### ✍️ Exercise: Write code to threshold `filtered_image` and generate a binary image assigned to the variable `binary_mask`

In [11]:
binary_mask = filtered_image > min_threshold

### ✍️ Exercise: Run the following code to view `binary_mask` with `ndv`

In [ ]:
ndv.imshow(binary_mask)

In [ ]:
viewer = ndv.imshow(binary_mask)

In [ ]:
viewer.widget().children[1].snapshot()

***

## 4. Mask Labeling

**Concept.**  
Now that we have a binary mask that has white, or value `True`, pixels that match the image foreground and black, or value  `False`, pixels that match the image background, we need a way to distinguish individual objects within this mask. **Labeling** a mask is when we identify individual objects within a binary mask and assign them a unique identifier. 

### Labeling a binary mask
From `skimage.measure` we can use `label()` to label a curated binary mask.

| Function | Description | How to import it | Documentation Link | 
|---------|---------|----------------| ----------------|
| `label()` | Label connected regions of an image for Instance segmentation | `from skimage.measure import label` | [threshold_otsu](https://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.label) |

Here's how we use `label` from `skimage.measure` to label a binary mask: 
```python
labeled_image = label(binary_mask)
```
Here, `label()` will take the inputted `binary_mask` and count each connected object in the image and assign them a number starting from 1. It will then return an image where each object's pixels have the value of its object's assigned number. It will return the transformed image, so we should assign it a variable, like `labeled_image`!

### ✍️ Exercise: Write code to label `binary_mask` and assign it to the variable `labeled_image`
Remember - we need to import the label function from skimage.measure in Setup!

In [12]:
labeled_image = label(binary_mask)

### Displaying a labeled mask on top of the image
Let's now summarize our final segmentation result in 1 image by viewing the `labeled_image` overlaid onto the original `raw_image`. From `skimage.color`, we can use `label2rgb` to do this. 

| Function | Description | How to import it | Documentation Link | 
|---------|---------|----------------| ----------------|
| `label2rgb()` | Returns an RGB image where color-coded labels are painted over the image | `from skimage.color import label2rgb` | [label2rgb](https://scikit-image.org/docs/dev/api/skimage.color.html#skimage.color.label2rgb) |

Here's how we can use `label2rgb` from `skimage.color` to summarize our segmentation result: 
```python
seg_summary = label2rgb(labeled_image, image = raw_image, bg_label=0)
```
Here, `label2rgb()` is filled with a few arguments: 
1. Argument 1: The labeled mask `labeled_image`
2. Argument 2: The original image we want the `labeled_image` overlaid onto, specified as `image = raw_image`
3. Argument 3: Background transparency of `labeled_image`, which is set to 0 by specifying `bg_label=0` 

The output will be an rgb image of the labeled mask overlaid onto the raw image. Assign this result a variable for easy reference, like `seg_summary`!

### ✍️ Exercise: Write code that creates an image of `labeled_image` overlaid onto `raw_image` and assign it to the variable `seg_summary`
Remember - we need to import the `label2rgb` function from `skimage.color` in Setup!

In [23]:
seg_summary = label2rgb(labeled_image, image=raw_image, bg_label=0)

/Users/edelase/HMS Dropbox/Eva de la Serna/Eva_CITE_folder/projects/bobiac/git/bobiac/.venv/lib/python3.12/site-packages/skimage/color/colorlabel.py:149: UserWarning: Negative intensities in `image` are not supported
  rgb = _label2rgb_overlay(


### ✍️ Exercise: Run the following code to view `seg_summary` with `ndv`

In [ ]:
ndv.imshow(seg_summary)

In [ ]:
viewer = ndv.imshow(seg_summary)

In [ ]:
viewer.widget().children[1].snapshot()

How does the segmentation result look? Are all labels corresponding to individual nuclei? If not, additional processing steps are needed to refine `binary_mask`.

***

## 5. Mask Refinement

**Concept.**  
**Mask refinement** is needed when a binary mask still does not accurately match the image foreground. In the context of our nuclei example image, we need to apply additional processing steps to remove connected objects that are too small to be nuclei, fill any holes within nuclei, and separate touching nuclei.

### Common mask refinement steps
There are many different ways we can refine a binary mask. The table below summarizes very common refinement steps:

| Algorithm Name | Description | How to import it | Documentation Link |
|---------|---------|----------------|----------------|
| Watershed Transform | A useful algorithm for separating touching objects. The output is a labeled image. | `from skimage.segmentation import watershed` | [skimage.segmentation.watershed](https://scikit-image.org/docs/0.25.x/api/skimage.segmentation.html#skimage.segmentation.watershed) |
| Remove Objects by Size | Remove objects smaller than the specified size from the foreground.  | `from skimage.morphology import remove_small_objects` | [skimage.morphology.remove_small_objects](https://scikit-image.org/docs/0.25.x/api/skimage.morphology.html#skimage.morphology.remove_small_objects) |
| Morphological Opening | Mathematical operation that results in small object removal | `from skimage.morphology import binary_opening` | [skimage.morphology.binary_opening](https://scikit-image.org/docs/stable/api/skimage.morphology.html#skimage.morphology.binary_opening) |
| Morphological Closing | Mathematical operation that results in small hole removal | `from skimage.morphology import binary_closing` | [skimage.morphology.binary_closing](https://scikit-image.org/docs/stable/api/skimage.morphology.html#skimage.morphology.binary_closing) |
| Morphological Erosion | Mathematical operation that reduces shape size | `from skimage.morphology import binary_erosion` | [skimage.morphology.binary_erosion](https://scikit-image.org/docs/stable/api/skimage.morphology.html#skimage.morphology.binary_erosion) |
| Morphological Dilation | Mathematical operation that increases shape size | `from skimage.morphology import binary_dilation` | [skimage.morphology.binary_dilation](https://scikit-image.org/docs/stable/api/skimage.morphology.html#skimage.morphology.binary_dilation) |

Let's now walk through steps to remove objects smaller than nuclei with `remove_small_objects()`, fill in any holes within nuclei with `binary_closing()`, and then separate touching nuclei with `watershed()`.

### Removing objects smaller than nuclei
In many cases, thresholding will be unsuccessful at rejecting image objects that are debris, as these tend to have high intensity values. However, we can use differences in object size to reject anything that is too small to be a nucleus. 

From `skimage.morphology` we can use the `remove_small_objects` function, which we already imported in Setup, to remove any connected objects of a specified `min_size`. Here is how we can do that:
```python
binary_mask_sized = remove_small_objects(binary_mask, min_size=10)
```
Here, `remove_small_objects()` will take the inputted `binary_mask` and set any connected object that is smaller than `min_size=10` to have `False` values (in other words, be rejected as foreground). It will then return the updated binary mask, so we should assign it to a variable, such as `binary_mask_sized`.

### ✍️ Exercise: Write code to remove objects smaller than nuclei in `binary_mask` and assign it to the variable `binary_mask_sized`
Remember, we need to import functions we want to use in Setup!

In [ ]:
binary_mask_sized = remove_small_objects(binary_mask, min_size=10)

### ✍️ Exercise: Run the code below to use `ndv` to display `binary_masked_sized`

In [ ]:
ndv.imshow(binary_mask_sized)

In [ ]:
viewer = ndv.imshow(binary_mask_sized)

In [ ]:
viewer.widget().children[1].snapshot()

### Filling holes within nuclei
While filtering helps reduce the effect of noise on thresholding, sometimes there will still be areas within an object that are below the minimum threshold value. These areas will show up as holes within a connected object. We can fill these holes by applying a morphological closing operation to the mask. 

From `skimage.morphology` we can use the `binary_closing` function, which we already imported in Setup, to fill small holes within nuclei. Here is how we can do that:
```python
binary_mask_filled = binary_closing(binary_mask_sized)
```
Here, `binary_closing()` will take the inputted `binary_mask_sized` and perform a morphological closing operation optimized for binary images. It will then return the updated binary mask, so we should assign it to a variable, such as `binary_mask_filled`.

### ✍️ Exercise: Write code to fill holes within nuclei in `binary_mask_sized`, and assign the updated mask to the variable `binary_masked_filled`
Remember, we need to import functions we want to use in Setup!

In [ ]:
binary_mask_filled = binary_closing(binary_mask_sized)

### ✍️ Exercise: Run the code below to use `ndv` to display `binary_masked_filled`

In [ ]:
ndv.imshow(binary_mask_filled)

In [ ]:
viewer = ndv.imshow(binary_mask_filled)

In [ ]:
viewer.widget().children[1].snapshot()

### Separating touching nuclei
Let's apply the Watershed Transformation to our `binary_mask` to separate any touching nuclei. From `skimage.segmentation`, we can use `watershed()` to do this. 

The `watershed()` function needs the following inputs:
1. The inverse of the distance transform of the binary mask
2. The seeds: labeled image of the peaks of the distance transform
3. The binary mask

We are going to need a few additional functions to provide the first two inputs to the `watershed()` function. 

| Function | Description | How to import it | Documentation Link |
|---------|---------|----------------|----------------|
| `distance_transform_edt()` | Calculates the distance transform of the input | `from scipy.ndimage import distance_transform_edt` | [scipy.ndimage.distance_transform_edt](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.distance_transform_edt.html) |
| `peak_local_max` | Remove objects smaller than the specified size from the foreground.  | `from skimage.feature import peak_local_max` | [skimage.feature.peak_local_max](https://scikit-image.org/docs/0.25.x/api/skimage.feature.html#skimage.feature.peak_local_max) |

#### Computing the distance transform
We can use the `distance_transform_edt()` function from `scipy.ndimage` to get the distance transform of our refined binary_mask `binary_mask_filled`:
```python
# compute the distance transform
distance_transform = distance_transform_edt(binary_mask_filled)
```

#### Finding the coordinates of peaks in the distance transform
Once we have the distance transform of `binary_mask_filled`, we can use the `peak_local_max()` function from `skimage.feature` to get local maxima values from the distance transform. However, we want to make sure that we get only 1 local maximum per object. We therefore can apply a `footprint` input confine a region the `peak_local_max` function will look for local maxima. Doing so will constrain how many maxima the function returns. We can also specify a `min_distance` separating peaks, which will also help constrain the number of maxima to be 1 per nucleus. 

Here's how we would write the code:
``` python
# find local maxima coordinates in the distance transform
local_maxima_coords = peak_local_max(distance, footprint=np.ones((25, 25)), min_distance=10)
```

Now that we have the maxima, we need to organize them for the `watershed()` function as a labeled image. We can do that by creating a binary image from the local maxima coordinates and then labeling the mask:
```python
# create a binary image from the local maxima coordinates
local_maxima = np.zeros_like(binary_mask_filled, dtype=bool)
local_maxima[tuple(local_maxima_coords.T)] = True
# use the local maxima to create seeds for the watershed algorithm
seeds = label(local_maxima)
```

#### Applying the Watershed Transform

``` python
# apply the watershed algorithm to segment the image and get labels
labeled_ws_image = watershed(-distance_transform, seeds, mask=binary_mask_filled)
```

Here, `watershed()` will apply the Watershed Transform to the inputted `binary_image_filled`. It will return the transformed, labeled image, so we should assign it a variable, such as `labeled_ws_image`!

### ✍️ Exercise: Write code to apply a watershed transform to `binary_mask_filled` and assign it to the variable `labeled_ws_image`
Remember, all packages have already been imported in Setup!

In [ ]:
# compute the distance transform
distance_transform = distance_transform_edt(binary_mask_filled)

# find local maxima coordinates in the distance transform
local_maxima_coords = peak_local_max(
    distance_transform, footprint=np.ones((25, 25)), min_distance=10
)

# create a binary image from the local maxima coordinates
local_maxima = np.zeros_like(binary_mask_filled, dtype=bool)
local_maxima[tuple(local_maxima_coords.T)] = True
# use the local maxima to create seeds for the watershed algorithm
seeds = label(local_maxima)

# apply the watershed algorithm to segment the image and get labels
labeled_ws_image = watershed(-distance_transform, seeds, mask=binary_mask_filled)

### ✍️ Exercise: Write code that creates an image of `labeled_ws_image` overlaid onto `raw_image` and assign it to the variable `seg_summary_refined`

In [ ]:
seg_summary_refined = label2rgb(labeled_ws_image, image=raw_image, bg_label=0)

### ✍️ Exercise: Run the code below to use `ndv` to display `seg_summary_refined`

In [ ]:
ndv.imshow(seg_summary_refined)

In [ ]:
viewer = ndv.imshow(seg_summary_refined)

In [ ]:
viewer.widget().children[1].snapshot()

## END OF FIRST LAB SECTION - STOP HERE FOR LAST LECTURE COMPONENT!

***

## 6. Processing Many Images

**Concept.**  
Statistically relevant & reproducible measurements come from analyzing many fluorescence images. Therefore, we need to adapt our code to efficiently run on many images, not just 1 at a time! We can do so by implementing a `for` loop and the `Path` function to our image path handling. 

### Using a for loop to apply processing steps to many images
From `pathlib`, we can use `Path` in conjunction with a `for` loop to iterate through many image file paths. Here's how we would write the code to do that:
```python
from pathlib import Path
folder_path = Path(“/Users/edelase/bobiac”) # update with your folder's path
for image_path in path.iterdir():
    # do our classic segmentation processing steps
```

### Decomposing our processing steps into functions
We could just copy paste the lines of code that we wrote in the exercises above one after another within a `for` loop, but a better strategy would be to organize the code for each processing step into its own function. Doing so will make each step easier to reuse in future code you write or share with collaborators.

### ✍️ Exercise: Improve the code below! 
1. Group processing steps into functions
2. Add a for loop so processing steps can be applied to many images

- load an image from ...
- filter image
- generate the ...

In [ ]:
# specify dependencies

# load the image
image_path = ""
raw_image = tifffile.imread(image_path)

# filter the image with gaussian filter
filtered_image = gaussian(raw_image)

# threshold filtered_image to generate binary mask
binary_mask = filtered_image > threshold_otsu(filtered_image)

# Remove small objects
binary_mask_sized = remove_small_objects(binary_mask, min_size=10)

# Fill small holes by performing morphological closing
binary_mask_filled = binary_closing(binary_mask_sized)

# apply watershed to separate nuclei and label mask
# compute the distance transform
distance_transform = distance_transform_edt(binary_mask_filled)
# find local maxima coordinates in the distance transform
local_maxima_coords = peak_local_max(
    distance_transform, footprint=np.ones((25, 25)), min_distance=10
)
# create a binary image from the local maxima coordinates
local_maxima = np.zeros_like(binary_mask_filled, dtype=bool)
local_maxima[tuple(local_maxima_coords.T)] = True
# use the local maxima to create seeds for the watershed algorithm
seeds = label(local_maxima)
# apply the watershed algorithm to segment the image and get labels
labeled_ws_image = watershed(-distance_transform, seeds, mask=binary_mask_filled)

# display resulting labeled_image overlaid onto raw_image
seg_summary = label2rgb(labeled_ws_image, image=raw_image, bg_label=0)